In [1]:
import os

import matplotlib
import sklearn
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from imblearn.under_sampling import RepeatedEditedNearestNeighbours, ClusterCentroids
from imblearn.under_sampling import InstanceHardnessThreshold
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
import math
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectFromModel, chi2, SelectKBest

In [2]:
text_neg = []
text_pos = []


def loaddata(path):
    numwords = []
    file = open(path, encoding="utf-8")
    texts_projects = []
    label_projects = []
    for line in file.readlines():
        line = line.strip("\n")
        a = line[13:].lower()
        if a == '':
            continue
        if a == '   ':
            continue
        if a == ' ':
            continue
        if a == '  ':
            continue
        texts_projects.append(a)

        counter = len(a.split(" "))
        numwords.append(counter)
        if line[4:12] == "negative":
            label_projects.append(0)
            text_neg.append(a)
        else:
            label_projects.append(1)
            text_pos.append(a)
    return texts_projects, label_projects, numwords

In [3]:
def dataSplit(target):
    path = './data'
    files = os.listdir(path)
    texts = []
    labels = []
    numwords = []
    text1 = []
    label1 = []
    numwords1 = []
    for file in files:
        if file != target:
            # print(file)
            text, label, numword = loaddata(r'./data/' + file)
            texts += text
            labels += label
            numwords += numword
    text1, label1,numwords1 = loaddata(r'./data/' + target)
    return texts, labels, numwords, text1, label1, numwords1

In [4]:
path = './data'
files = os.listdir(path)
ft = 0
for target in files:

    x_tr, y_train, numwords, x_te, y_test, numwords1 = dataSplit(target)
    idf = TfidfVectorizer(max_features = 10000, lowercase=False)

    x_train = idf.fit_transform(x_tr)
    x_test = idf.transform(x_te)
    smo = BorderlineSMOTE()
    x_train, y_train = smo.fit_resample(x_train, y_train)
    smo.fit(x_train, y_train)
    from lightgbm import LGBMClassifier
    # gbm0=LGBMClassifier(max_depth= 4, num_leaves= 50,learning_rate=0.1, n_estimators=800)
    gbm0=LGBMClassifier(max_depth= 3, num_leaves= 50,learning_rate=0.1, n_estimators=800)
    eval_set = [(x_test, y_test)]
    gbm0.fit(x_train, y_train, eval_set=[(x_test, y_test)])
    y_predict = gbm0.predict(x_test)
    y_pred=gbm0.predict(x_test)
    y_predprob=gbm0.predict_proba(x_test)[:,1]            #样本预测为类别1的概率（默认使用正样本标签计算AUC）

    tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_predict).ravel()
    rec = tp / (tp + fn)
    Recall = tp / (tp + fn)
    specificity = tn / (fp + tn)
    precision = tp / (tp + fp)
    gmeans = math.sqrt(specificity * rec)
    f1 = 2 * precision * rec / (precision + rec)
    print("recall:", rec)

    print("precision:", precision)
    print("f1:", f1)
    ft += f1
    print(y_pred)
print("avg f1:",ft/10)

[1]	valid_0's binary_logloss: 0.628938
[2]	valid_0's binary_logloss: 0.575516
[3]	valid_0's binary_logloss: 0.530499
[4]	valid_0's binary_logloss: 0.492187
[5]	valid_0's binary_logloss: 0.459322
[6]	valid_0's binary_logloss: 0.43095
[7]	valid_0's binary_logloss: 0.405232
[8]	valid_0's binary_logloss: 0.384295
[9]	valid_0's binary_logloss: 0.364177
[10]	valid_0's binary_logloss: 0.347916
[11]	valid_0's binary_logloss: 0.331679
[12]	valid_0's binary_logloss: 0.3189
[13]	valid_0's binary_logloss: 0.305915
[14]	valid_0's binary_logloss: 0.295683
[15]	valid_0's binary_logloss: 0.28539
[16]	valid_0's binary_logloss: 0.275976
[17]	valid_0's binary_logloss: 0.268209
[18]	valid_0's binary_logloss: 0.262137
[19]	valid_0's binary_logloss: 0.255385
[20]	valid_0's binary_logloss: 0.249299
[21]	valid_0's binary_logloss: 0.244337
[22]	valid_0's binary_logloss: 0.239368
[23]	valid_0's binary_logloss: 0.235777
[24]	valid_0's binary_logloss: 0.232739
[25]	valid_0's binary_logloss: 0.228796
[26]	valid_0'